## YOLOv8

In [1]:
# Mount Google Drive

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Checking how many images are in a folder
import os
total_images = "/content/gdrive/MyDrive/thebigthree/data/images"
print(len(os.listdir(total_images)))
num_images = "/content/gdrive/MyDrive/thebigthree/data/labels"
print(len(os.listdir(num_images)))

1182
1182


In [ ]:
# !pip install ultralytics
!git clone https://github.com/ultralytics/ultralytics
# Navigate to the cloned directory
%cd ultralytics
# Install the package in editable mode for development
%pip install -e .

In [ ]:
### 4. Train model ###

import os

from ultralytics import YOLO

# Load a model
model = YOLO("yolov8s.pt")  # load pre trained model

# Defining args
data= '/content/gdrive/MyDrive/3dPrinterFault/3d_printer_fault_v8.yaml'
optimizer= 'AdamW'
epochs=100
imgsz=640
batch=25
weight_decay=0.0005
cls=0.7
lr0=0.001
lrf=0.001

# Use the model
!yolo task=detect mode=train model=yolov8n.pt optimizer=AdamW data='/content/gdrive/MyDrive/dataset-after-aug/data.yaml' epochs=150 imgsz=640 batch=40 weight_decay=0.0005 save=True cls=0.7 lr0=0.001 lrf=0.001


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('/content/runs/detect/train/confusion_matrix.png') #Replace "image.jpg" with the path of your image
plt.imshow(img)
plt.axis('off')
plt.show()

In [11]:
import locale
def getpreferredencoding(do_setlocale = True):
      return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# Copy results to drive

!scp -r /content/ultralytics/runs '/content/gdrive/MyDrive/runs/runs2'

# KFolds Validation with YOLOv8

In [ ]:
# Provided by https://docs.ultralytics.com/guides/kfold-cross-validation/#generating-feature-vectors-for-object-detection-dataset

import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold

savedrivepath = Path('/content/gdrive/MyDrive/thebigthree/after')
dataset_path = Path('/content/gdrive/MyDrive/thebigthree/data') # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt")) # all data in 'labels'
# print(len(labels))

yaml_file = '/content/gdrive/MyDrive/thebigthree/thebigthree.yaml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = sorted(classes.keys())
# print(cls_idx)

indx = [l.stem for l in labels] # uses base filename as ID (no extension)

labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

# Counting instance of each class-label
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1
        # print(lbl_counter)


    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

# print(labels_df.shape)
# print(labels_df)

ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))
# print(kfolds)
folds = [f'split_{n}' for n in range(1, ksplit + 1)]

folds_df = pd.DataFrame(index=indx, columns=folds)
# print(folds_df)
# print(folds_df.to_string())

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)
# print(labels_df)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio


In [7]:
supported_extensions = ['.jpg', '.jpeg', '.png','.JPG']

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images').rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(savedrivepath / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val_test2')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

In [8]:
for image, label in zip(images, labels):
     for split, k_split in folds_df.loc[image.stem].items():

        if image.stem not in folds_df.index:
          raise ValueError(f"Stem {image.stem} not found in folds_df index")
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [ ]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

## YOLOv8 Training with Kfolds

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt", task='detect')  # load pre trained model

results = {}

batch = 16
project = 'kfold_demo'
epochs=100
cls=0.7
lr0=0.001
lrf=0.001
optimizer='AdamW'

for k in range(ksplit):
  dataset_yaml = ds_yamls[k]
  model.train(data=dataset_yaml, epochs=epochs, batch=batch, project=project, cls=cls, lr0=lr0, lrf=lrf, optimizer=optimizer)
  results[k] = model.metrics

In [12]:
!scp -r /content/ultralytics/kfold_demo '/content/gdrive/MyDrive/thebigthree/runs'

### Information

- Confusion Matrix Info: https://www.v7labs.com/blog/confusion-matrix-guide
- Meanings of Different Confidence Levels: https://help.nightfall.ai/nightfall-ai/detection-engine/detection-engine-faqs/what-do-different-confidence-levels-mean
- Mean Average Precision (mAP) Info: https://www.v7labs.com/blog/mean-average-precision

Credit to: https://www.kaggle.com/datasets/mikulhe/3d-printing-errors for the "good" print dataset

In [ ]:
import os
from pathlib import Path

good_photos_dir = Path('/content/gdrive/MyDrive/thebigtwo/good_images/images')
good_phots_txt = Path('/content/gdrive/MyDrive/thebigtwo/good_images/labels')

files = os.listdir(good_photos_dir)

for file in files:
  # Extract name of image without .jpg
  image_name, _ = os.path.splitext(file)
  # Creates a text file with the name of that image
  text_file_path = os.path.join(good_phots_txt, image_name + '.txt')
  with open(text_file_path, 'w') as text_file:
      text_file.write('')